# chat UI in fastHTML

> chat UI implemented in fastHTML

In [118]:
#| default_exp chat_ui

In [119]:
#| hide
from nbdev.showdoc import *

## Response generation

For our first MVP, response generation mostly concern with GPT models answering generic questions and using a single tool for capturing and extracting information from a Youtube Livestream.

In [120]:
#| export
# Importing openai and our custom functions
import openai
import json
import ast
import inspect

from typing import Optional, Union
from types import NoneType
from llmcam.ytlive import capture_youtube_live_frame
from llmcam.gpt4v import ask_gpt4v

### `extract_live_info` Function

In [121]:
#| export
def extract_live_info(
        link: Optional[str] = None  # YouTube Live link
    ) -> dict:  # The extracted information
    """Extarct information from a YouTube Live"""
    if link:
        image = capture_youtube_live_frame(link)
    else:
        image = capture_youtube_live_frame()
    
    return ask_gpt4v(image)

One additional utility to extract parameter descriptions from a funcion:

In [122]:
#| export
# Extract parameter comments from the function
def extract_parameter_comments(func):
    """Extract comments for function arguments"""
    # Get the source code of the function
    source = inspect.getsource(func)
    # Parse the source code into an AST
    tree = ast.parse(source)
    
    # Extract comments for function arguments
    comments = {}
    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef) and node.name == func.__name__:
            # Get arguments and comments in the function
            for arg in node.args.args:
                arg_name = arg.arg
                # Check if there's an inline comment associated with the argument
                if arg.end_lineno and arg.col_offset:
                    # Loop through the source code lines to find the comment
                    lines = source.splitlines()
                    for line in lines:
                        if line.strip().startswith(f"{arg_name}:") and "#" in line:
                            comment = line.split("#")[1].strip()
                            comments[arg_name] = comment
    return comments

Test usage with extract live function:

In [123]:
# Extract parameter comments
param_comments = extract_parameter_comments(extract_live_info)
print(param_comments)

{'link': 'YouTube Live link'}


In [124]:
#| hide
# Test the function
assert param_comments['link'] == "YouTube Live link"

### Utilities for GPT Function calling

We can use dynamic utilities functions to integrate this to GPT Function calling:  

- Parameter converter: convert Python parameter types into schema accepted formats
- Schema generator: extract function information into tool schema to bet set for GPT
- Function execution: execute function dynamically based on function names and input arguments

In [125]:
#| export
def param_converter(
        param_type,  # The type of the parameter
        description  # The description of the parameter
    ) -> dict:  # The converted parameter
    """Convert parameter types to acceptable types for tool schema"""
    simple_types = {
        str: "string",
        int: "number",
        float: "number",
        bool: "boolean",
    }
    if param_type in simple_types:
        return { "type": simple_types[param_type], "description": description }
    elif param_type == NoneType:
        return { "type": "null", "description": "A default value will be automatically used." }
    
    if hasattr(param_type, '__origin__') and param_type.__origin__ == Union:
        # Recursively convert the types
        descriptions = description.split(" or ")
        subtypes = param_type.__args__
        if len(subtypes) > len(descriptions):
            descriptions = descriptions + ["A description is not provided"] * (len(subtypes) - len(descriptions))

        return {
            "anyOf": [param_converter(subtype, desc) for subtype, desc in zip(subtypes, descriptions)]
        }
    return { "type": "string", "description": description }

Test usage:

In [126]:
param_schema = param_converter(Optional[str], "YouTube Live link")
print(json.dumps(param_schema, indent=2))

{
  "anyOf": [
    {
      "type": "string",
      "description": "YouTube Live link"
    },
    {
      "type": "null",
      "description": "A default value will be automatically used."
    }
  ]
}


In [127]:
#| hide
# Test the function
assert param_schema == { "anyOf": [
        {
            "type": "string",
            "description": "YouTube Live link"
        },
        {
            "type": "null",
            "description": "A default value will be automatically used."
        }
    ]
}

In [128]:
#| export
def tool_schema(func):
    """Automatically generate a schema from its parameters and docstring"""
    # Extract function name, docstring, and parameters
    func_name = func.__name__
    func_description = func.__doc__ or "No description provided."
    signature = inspect.signature(func)
    
    # Create parameters schema
    parameters_schema = {
        "type": "object",
        "properties": {},
        "required": []
    }
    
    # Populate properties and required fields
    param_comments = extract_parameter_comments(func)
    for param_name, param in signature.parameters.items():
        param_type = param.annotation if param.annotation != inspect._empty else str
        
        # Add parameter to schema
        parameters_schema["properties"][param_name] = param_converter(
            param_type, 
            param_comments.get(param_name, "No description provided.")
        )
        
        # Mark as required if no default
        if param.default == inspect.Parameter.empty:
            parameters_schema["required"].append(param_name)
    
    # Build final tool schema
    tool_schema = [
        {
            "type": "function",
            "function": {
                "name": func_name,
                "description": func_description,
                "parameters": parameters_schema,
            }
        }
    ]
    return tool_schema

In [152]:
#| export
tools = tool_schema(extract_live_info)

In [154]:
#| echo: false
print(json.dumps(tools, indent=2))

[
  {
    "type": "function",
    "function": {
      "name": "extract_live_info",
      "description": "Extarct information from a YouTube Live",
      "parameters": {
        "type": "object",
        "properties": {
          "link": {
            "anyOf": [
              {
                "type": "string",
                "description": "YouTube Live link"
              },
              {
                "type": "null",
                "description": "A default value will be automatically used."
              }
            ]
          }
        },
        "required": []
      }
    }
  }
]


In [130]:
#| hide
# Test the function
assert tools[0]["function"]["name"] == "extract_live_info"
assert tools[0]["function"]["description"] == "Extarct information from a YouTube Live"

In [131]:
#| export
# Support functions to handle tool response,where res == response.choices[0].message
def fn_name(res): return res.tool_calls[0].function.name
def fn_args(res): return json.loads(res.tool_calls[0].function.arguments)    
def fn_exec(res): return globals().get(fn_name(res))(**fn_args(res))
def fn_result_content(res):
    """Create a content containing the result of the function call"""
    content = dict()
    content.update(fn_args(res))
    content.update({fn_name(res): fn_exec(res)})
    return json.dumps(content)

### Response generator with GPT

In [132]:
#| export
def generate_response(
        message: str,  # New message frorm the user
        history : list,  # Previous messages
    ) -> list:  # The responses of GPT model
    """Generate a response from the messages"""
    responses = []

    def complete(role, content, tool_call_id=None):
        "Send completion request with messages, and save the response in messages again"
        responses.append({
            "role":role, 
            "content":content, 
            "tool_call_id":tool_call_id
        })
        response = openai.chat.completions.create(
            model="gpt-4o", 
            messages=history + responses, 
            tools=tools
        )
        res = response.choices[0].message
        responses.append(res.to_dict())
        if res.to_dict().get('tool_calls'):
            complete(role="tool", content=fn_result_content(res), tool_call_id=res.tool_calls[0].id)

    complete("user", message)

    return responses

In [ ]:
#| eval: false
example = generate_response(
    "Yes, please use the default link.",
    [
        {
            "role": "user",
            "content": "Extract information from a YouTube Live",
        },
        {
            "content": "Please provide the YouTube Live link you would like to extract information from, or let me know if you'd like to use a default link.",
            "refusal": None,
            "role": "assistant"
        }
    ])
print(json.dumps(example, indent=2))

[youtube] Extracting URL: https://www.youtube.com/watch?v=LMZQ7eFhm58
[youtube] LMZQ7eFhm58: Downloading webpage
[youtube] LMZQ7eFhm58: Downloading ios player API JSON
[youtube] LMZQ7eFhm58: Downloading mweb player API JSON
[youtube] LMZQ7eFhm58: Downloading m3u8 information
[youtube] LMZQ7eFhm58: Downloading m3u8 information
[
  {
    "role": "user",
    "content": "Yes, please use the default link.",
    "tool_call_id": null
  },
  {
    "content": null,
    "refusal": null,
    "role": "assistant",
    "tool_calls": [
      {
        "id": "call_WNKVSnuYqt9X86YZfAfa7lj6",
        "function": {
          "arguments": "{\"link\":null}",
          "name": "extract_live_info"
        },
        "type": "function"
      }
    ]
  },
  {
    "role": "tool",
    "content": "{\"link\": null, \"extract_live_info\": {\"timestamp\": \"2024-01-11T01:15:45\", \"location\": null, \"dimensions\": {\"width\": 1280, \"height\": 720}, \"buildings\": {\"number_of_buildings\": 15, \"building_height_ran

## Chat App initialization

Start by creating the chat application with `FastHTML`.

In [134]:
#| export
import uvicorn
import importlib.util
from fasthtml.common import *

In [135]:
#| export
# Set up the app, including daisyui and tailwind for the chat component
hdrs = (picolink, Script(src="https://cdn.tailwindcss.com"),
        Link(rel="stylesheet", href="https://cdn.jsdelivr.net/npm/daisyui@4.11.1/dist/full.min.css"))
app = FastHTML(hdrs=hdrs)

## Chat components

Basic chat UI components can include Chat Message and a Chat Input. For a Chat Message, the important attributes are the actual message (str) and the role of the message owner (user - boolean value whether the owner is the user, not the AI assistant).

In [136]:
#| export
# Chat message component (renders a chat bubble)
def ChatMessage(
        msg: str,  # Message to display
        user: bool  # Whether the message is from the user or assistant
    ):  # Returns a Div containing the chat bubble
    paragraphs = msg.split("\n")
    # Set class to change displayed style of bubble
    bubble_class = "chat-bubble-primary" if user else 'chat-bubble-secondary'
    chat_class = "chat-end" if user else 'chat-start'
    return  Div(cls=f"chat {chat_class}")(
                Div('User' if user else 'Assistant', cls="chat-header"),
                Div(
                    *[P(p, style=f"color: {'black' if user else 'white'};") for p in paragraphs], 
                    cls=f"chat-bubble {bubble_class}", 
                    style=f"background-color: {'#03fcad' if user else '#025238'};"),
                Hidden(msg, name="messages"),  # Hidden field for submitting past messages to form
                Hidden("user" if user else "assistant", name="roles")  # Hidden field for submitting corresponding owners
            )

In [137]:
show_doc(ChatMessage)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/chat_ui.py#L21){target="_blank" style="float:right; font-size:smaller"}

### ChatMessage

>      ChatMessage (msg:str, user:bool)

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| msg | str | Message to display |
| user | bool | Whether the message is from the user or assistant |

For the chat input, set the name for submitting a new message via form.

In [138]:
#| export
# The input field for the user message. Also used to clear the
# input field after sending a message via an OOB swap
def ChatInput():  # Returns an input field for the user message
    return Input(name='msg', id='msg-input', placeholder="Type a message",
                 cls="input input-bordered w-full rounded-l-2xl bg-stone-800", 
                 hx_swap_oob='true'  # Re-render the element to remove submitted message
                )

In [139]:
show_doc(ChatInput)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/chat_ui.py#L39){target="_blank" style="float:right; font-size:smaller"}

### ChatInput

>      ChatInput ()

### Action Buttons

Simple actions for creating a new message from the user side.

In [140]:
#| export
def ActionButton(
        content: str  # Text to display on the button
    ):  # Returns a button with the given content

    return Form(
        hx_post="/",
        hx_target="#chatlist",
        hx_swap="beforeend",  # Location: just before the end of element
    )(
        Hidden(content, name="msg"),
        Button(content, cls="btn btn-secondary")
    )

def ActionPanel():  # Returns a panel of action buttons
    return Div(
        ActionButton("Do something"),
        ActionButton("A different action"),
        cls="flex flex-row h-fit px-24 gap-4 pt-4"
    )

In [141]:
show_doc(ActionButton)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/chat_ui.py#L46){target="_blank" style="float:right; font-size:smaller"}

### ActionButton

>      ActionButton (content:str)

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| content | str | Text to display on the button |

In [142]:
show_doc(ActionPanel)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/chat_ui.py#L56){target="_blank" style="float:right; font-size:smaller"}

### ActionPanel

>      ActionPanel ()

## Router

### Home page
The home page should contain our message list and the Chat Input. The main page can be extracted by accessing the index (root) endpoint.

In [143]:
#| export
@app.get('/')
def index():
    sidebar = Div(
        H1("Conversations"),
        cls="w-[30vw] bg-stone-800"
    )
    page =  Div(cls="w-full flex flex-col p-0")(
        ActionPanel(),
        Form(
            hx_post="/",  # Operation: some POST endpoint with function `send` 
            hx_target="#chatlist",  # Target: element with ID 'chatlist'
            hx_swap="beforeend",  # Location: just before the end of element
            cls="w-full flex flex-col px-24 h-[90vh]"
        )(
            # The chat list
            Div(id="chatlist", cls="chat-box overflow-y-auto flex-1 w-full mt-10")(
                # One initial message from AI assistant
                ChatMessage("Hello! I'm a chatbot. How can I help you today?", False),
            ),
            # Input form
            Div(cls="h-fit mb-5 mt-5 flex space-x-2 mt-2")(
                Group(
                    ChatInput(), 
                    Button("Send", cls="btn btn-primary rounded-r-2xl", style="background-color: #03fcad;"))
            )
        )   
    )
    return Main(
        sidebar,
        page, 
        data_theme="forest", 
        cls="h-[100vh] w-full relative flex flex-row items-stretch overflow-hidden transition-colors z-0 p-0",)

### Form submission

At submission, this function should:

- Extract the new and all previous chat history  
- Prompt & get answers from ChatGPT from all these messages  
- Return a new ChatMessage

In [ ]:
#| export

# Handle the form submission
@app.post('/')
def send(msg: str, messages: list[str] = None, roles: list[str] = None):
    if not messages: messages = []
    if not roles: roles = []

    history = [ {"role": role, "content": message} for role, message in zip(roles, messages) ]
    
    # Add the user's message to the chat history
    responses = generate_response(msg, history)
    chat_messages = [
        ChatMessage(res['content'], res['role'] == 'user') for res in responses if 'content' in res \
            if res['role'] in ['user', 'assistant'] and res['content'] is not None
    ]
    
    return (*chat_messages,
            ChatInput()) # And clear the input field via an OOB swap

[youtube] Extracting URL: https://www.youtube.com/watch?v=LMZQ7eFhm58
[youtube] LMZQ7eFhm58: Downloading webpage
[youtube] LMZQ7eFhm58: Downloading ios player API JSON
[youtube] LMZQ7eFhm58: Downloading mweb player API JSON
[youtube] LMZQ7eFhm58: Downloading m3u8 information
[youtube] LMZQ7eFhm58: Downloading m3u8 information
[youtube] Extracting URL: https://www.youtube.com/watch?v=LMZQ7eFhm58
[youtube] LMZQ7eFhm58: Downloading webpage
[youtube] LMZQ7eFhm58: Downloading ios player API JSON
[youtube] LMZQ7eFhm58: Downloading mweb player API JSON
[youtube] LMZQ7eFhm58: Downloading m3u8 information
[youtube] LMZQ7eFhm58: Downloading m3u8 information


## Runner

In addition to the main app, an utility function is implemented to run the app just by importing and executing this function to a Python file.

In [145]:
#| export
def llmcam_chatbot(
        package_name="ninjalabo.llmcam",  # The installed package name
        module_name="chat_ui",  # The module containing the FastAPI app
        app_variable="app",  # The FastAPI app variable name
        host="0.0.0.0",  # The host to listen on
        port=5001,  # The port to listen on
        **uvicorn_kwargs  # Additional keyword arguments for uvicorn
    ):
    "Find and run the FastAPI app in the specified module within the given package."
    # Construct the full module path (e.g., 'llmcam.chat_ui')
    full_module_path = f"{package_name.split('.')[-1]}.{module_name}"

    # Check if the module exists in the installed package
    try:
        spec = importlib.util.find_spec(full_module_path)
        if spec is None:
            print(f"Module '{full_module_path}' not found in package '{package_name}'.")
            return
        # Dynamically run the Uvicorn server
        uvicorn.run(f"{full_module_path}:{app_variable}", host=host, port=port, **uvicorn_kwargs)
    except Exception as e:
        print(f"Error running the app: {e}")

In [146]:
show_doc(llmcam_chatbot)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/chat_ui.py#L118){target="_blank" style="float:right; font-size:smaller"}

### llmcam_chatbot

>      llmcam_chatbot (package_name='ninjalabo.llmcam', module_name='chat_ui',
>                      app_variable='app', host='0.0.0.0', port=5001,
>                      **uvicorn_kwargs)

*Find and run the FastAPI app in the specified module within the given package.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| package_name | str | ninjalabo.llmcam | The installed package name |
| module_name | str | chat_ui | The module containing the FastAPI app |
| app_variable | str | app | The FastAPI app variable name |
| host | str | 0.0.0.0 | The host to listen on |
| port | int | 5001 | The port to listen on |
| uvicorn_kwargs |  |  |  |

For running while testing with Jupyter notebook, use the `JupyUvi` in `fasthtml` to run in separate thread.

In [151]:
#| eval: false
from fasthtml.jupyter import *

server = JupyUvi(app=app)

In [150]:
server.stop()

In [148]:
#| hide
import nbdev; nbdev.nbdev_export()